# **Araç Fiyat Tahminleme**

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [19]:
data = pd.read_csv("ikinciyeni_veriseti.csv")
data.sample(10)

,Ekleme tarihi,Açılış fiyatı,Kapanış fiyatı,Kaput,SoLÖnÇamurluk,SoLÖnKapı,SoLArkaKapı,SoLArkaÇamurluk,bagaj,SaĞÖnÇamurluk,...,Alarm,4 X 4,Park Asistanı,CD Değiştiricili Radyo,Vakumlu Kapı,3 Araç Anahtarı Mevcut (Toplam),Kapı Hava Yastığı,Radyo - Kasetçalar,Merkezi Kilit,Döşeme: Suni Deri
319,2018-01-03,99.0,99.300,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
267,2018-01-15,80.0,TEKLİF YOK,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
280,2018-01-10,45.0,72.700,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
50,2018-02-05,91.0,TEKLİF YOK,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
366,2017-12-21,62.5,62.800,B,D,D,B,B,-,D,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
312,2018-01-03,67.5,TEKLİF YOK,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
106,2018-01-31,29.0,44.600,-,B,D,-,B,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
88,2018-02-01,60.0,87.500,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
131,2018-01-30,45.0,74.600,-,B,B,B,-,-,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
97,2018-02-01,78.0,78.600,-,-,-,-,-,-,-,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK


In [20]:
data.shape # Toplam satır ve sütun sayısı

(379, 87)

In [21]:
# Numeric values to "TEKLİF VAR" string on Kapanış fiyatı column
data['Kapanış fiyatı'] = data['Kapanış fiyatı'].apply(lambda x: 'TEKLİF VAR' if x != 'TEKLİF YOK' else x)

# Gerekli olmayan sütunları çıkarma (tarih ve metin sütunları)
columns_to_drop = ['Ekleme tarihi', 'İlk Tescil Tarihi', 'Son Muayene Tarihi','Döşeme: Suni Deri','Merkezi Kilit','Radyo - Kasetçalar','Kapı Hava Yastığı','Vakumlu Kapı','Park Asistanı','Ok Gösterimli Navigasyon Cihazı']
data_cleaned = data.drop(columns=columns_to_drop) #data_cleaned: Sütunları çıkarılmış olan yeni veri çerçevesi.

# Aynı veriyi içeren sütunları tespit etme
nunique = data_cleaned.apply(pd.Series.nunique) #data_cleaned.apply(pd.Series.nunique): Her bir sütundaki benzersiz değerlerin sayısını hesaplar.
cols_to_drop = nunique[nunique == 1].index #nunique[nunique == 1].index: Tüm değerleri aynı olan (benzersiz değer sayısı 1 olan) sütunların isimlerini seçer.

# Bu sütunları çıkarma
data_cleaned = data_cleaned.drop(columns=cols_to_drop)

# "-" değerlerini NaN yapma
data_cleaned = data_cleaned.replace('-', np.nan)

# NaN değerlerini rastgele "B" veya "D" ile doldurma
data_cleaned = data_cleaned.apply(lambda x: x.fillna(np.random.choice(['B', 'D']) if x.dtype == 'object' else x))
#.apply(lambda x: ...): Her bir sütunu ayrı ayrı işleyen bir lambda fonksiyonu uygular.
#x.fillna(...): Her sütundaki eksik değerleri (NaN) belirli bir değerle doldurur.
#np.random.choice(['B', 'D']) if x.dtype == 'object' else x: Sütun tipi object ise, eksik değerleri rastgele olarak 'B' veya 'D' ile doldurur; değilse doldurma işlemi yapmaz.


In [22]:
data_cleaned.sample(10) #data_cleaned.sample(10) ifadesi, data_cleaned adlı pandas DataFrame'inden rastgele 10 satır seçmek anlamına gelir.

,Açılış fiyatı,Kapanış fiyatı,Kaput,SoLÖnÇamurluk,SoLÖnKapı,SoLArkaKapı,SoLArkaÇamurluk,bagaj,SaĞÖnÇamurluk,SaĞÖnKapı,...,Elektrik Ayarlı Koltuk,Diz Hava Yastığı,Döşeme: Deri,Hafızalı Koltuk,CD Çalarlı Radyo,Döşeme: Alcantara/Deri,Haritalı Navigasyon Cihazı,Alarm,4 X 4,CD Değiştiricili Radyo
131,45.0,TEKLİF VAR,B,B,B,B,D,B,B,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
74,30.0,TEKLİF VAR,B,B,B,B,D,B,D,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
238,29.0,TEKLİF VAR,B,B,B,B,D,B,D,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
194,74.0,TEKLİF VAR,B,D,B,B,D,B,D,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
219,79.0,TEKLİF VAR,D,D,B,B,B,B,D,B,...,VAR,YOK,YOK,VAR,YOK,YOK,YOK,YOK,YOK,YOK
81,99.3,TEKLİF YOK,B,D,B,B,D,B,D,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
190,56.5,TEKLİF YOK,B,D,B,B,D,B,D,B,...,YOK,YOK,YOK,YOK,VAR,YOK,YOK,YOK,YOK,YOK
353,74.5,TEKLİF YOK,B,D,B,B,D,B,B,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
287,79.0,TEKLİF VAR,B,D,B,B,D,B,D,B,...,VAR,YOK,YOK,VAR,YOK,VAR,YOK,YOK,YOK,YOK
322,67.5,TEKLİF YOK,B,D,B,B,D,B,D,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK


In [23]:
data.dtypes # Veri tipleri

Ekleme tarihi                       object
Açılış fiyatı                      float64
Kapanış fiyatı                      object
Kaput                               object
SoLÖnÇamurluk                       object
                                    ...   
3 Araç Anahtarı Mevcut (Toplam)     object
Kapı Hava Yastığı                   object
Radyo - Kasetçalar                  object
Merkezi Kilit                       object
Döşeme: Suni Deri                   object
Length: 87, dtype: object

In [24]:
X = data_cleaned.drop(columns=['Kapanış fiyatı'])
y = data_cleaned['Kapanış fiyatı']

In [25]:
# Kategorik sütunları one-hot encoding ile dönüştürme
categorical_features = X.select_dtypes(include=['object']).columns.tolist() #categorical_features: X veri çerçevesindeki kategorik sütunların isimlerini içeren liste.
numerical_features = X.select_dtypes(include=['number']).columns.tolist() #numerical_features: X veri çerçevesindeki sayısal sütunların isimlerini içeren liste.

# ColumnTransformer ile sayısal ve kategorik sütunları dönüştürme
preprocessor = ColumnTransformer( #ColumnTransformer: Farklı sütunlar için farklı dönüştürme işlemlerini uygulamaya yarayan bir sklearn aracı.
    transformers=[
        ('num', StandardScaler(), numerical_features), #'num', StandardScaler(), numerical_features: Sayısal sütunları standartlaştırmak için StandardScaler kullanılır.
        ('cat', OneHotEncoder(), categorical_features) #'cat', OneHotEncoder(), categorical_features: Kategorik sütunları one-hot encoding ile dönüştürmek için OneHotEncoder kullanılır.
    ])

# Pipeline ile dönüşüm işlemlerini gerçekleştirme
pipeline = Pipeline(steps=[ #Pipeline: Veriyi işlemden geçirmek için adımların sırayla uygulanmasını sağlayan bir sklearn aracı.
    ('preprocessor', preprocessor) #steps=[('preprocessor', preprocessor)]: Ön işlemciyi (preprocessor) pipeline'a ekler.
])

# Verileri dönüştürme ve eğitim-test setlerine ayırma
X_processed = pipeline.fit_transform(X) #pipeline.fit_transform(X): X verisini pipeline içindeki işlemlerden geçirerek dönüştürür.

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size = 0.3, random_state = 33) #Output'u "y", geriye kalanları "X" olarak ele aldık.
#X ve y verisini %70-%30 oranın train-test verisi olarak ayırır.

In [27]:
clf = MLPClassifier() # KNN Sınıflandırması
clf.fit(X_train, y_train) # Eğitim verilerinin, KNN algoritmasına entegre edilmesi
prediction = clf.predict(X_test) # Test verisinin tahminlenme işlemi
prediction

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


array(['TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF YOK', 'TEKLİF VAR',
       'TEKLİF YOK', 'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF YOK',
       'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF YOK',
       'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF YOK',
       'TEKLİF YOK', 'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF VAR', 'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF VAR',
       'TEKLİF YOK', 'TEKLİF VAR', 'TEKLİF VAR', 'TEKLİ

In [28]:
accuracy_score(y_test, prediction)

0.7105263157894737

In [29]:
cm = confusion_matrix(y_test, prediction) # Confusion Matris
print(cm)

[[52 17]
 [16 29]]
